# Pytorch Tutorial

基本的に，以下のPyTorch初心者向けの公式ドキュメントに記載のものをそのまま動かしたものです。

説明を付記しました。

Reference:

PyTorch Tutorial

http://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py

## PyTorchとは

- 深層学習研究のプラットフォーム
- numpyよりも優れた，GPU上での計算の仕組みを提供する

## はじめに

### Tensors

PyTorchの基本は`torch.Tensor`に含まれる`Tensor`型。

`Tensor`は`numpy.ndarray`に似ているが，`Tensor`はGPU上での計算において優れた型である。

In [2]:
from __future__ import print_function
import torch

***
初期化されていない（5, 3）行列を`Tensor`型で宣言すると：

In [3]:
x = torch.Tensor(5, 3)
x


-2.4642e-07  4.5637e-41 -2.4642e-07
 4.5637e-41         nan  2.2493e-40
 4.4721e+21  1.6647e-41  6.7262e-44
 0.0000e+00  6.7262e-44  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00
[torch.FloatTensor of size 5x3]

***
乱数で初期化した（5, 3）行列を`Tensor`型で宣言すると

In [4]:
x = torch.rand(5, 3)
x


 0.0411  0.8354  0.4958
 0.8536  0.0238  0.7862
 0.0440  0.2469  0.2528
 0.6852  0.2259  0.2360
 0.2849  0.8614  0.7844
[torch.FloatTensor of size 5x3]

***
サイズの確認は：

In [5]:
x.size()

torch.Size([5, 3])

***

### Operations：演算

四則演算について，様々な表記がある。

例えば,加算についてみていく。

加算方法１：

In [7]:
y = torch.rand(5, 3)
x + y


 0.4565  1.7649  0.5953
 1.7001  0.1802  0.9959
 1.0320  0.7855  0.3711
 1.2973  0.9961  0.7011
 0.8641  1.3341  1.1177
[torch.FloatTensor of size 5x3]

***
加算方法２：

In [8]:
print(x + y)


 0.4565  1.7649  0.5953
 1.7001  0.1802  0.9959
 1.0320  0.7855  0.3711
 1.2973  0.9961  0.7011
 0.8641  1.3341  1.1177
[torch.FloatTensor of size 5x3]



***
加算方法３：

In [10]:
result = torch.Tensor(5, 3)      # make empty (5, 3) Tensor
torch.add(x, y, out = result)    # input x + y to Tensor
result


 0.4565  1.7649  0.5953
 1.7001  0.1802  0.9959
 1.0320  0.7855  0.3711
 1.2973  0.9961  0.7011
 0.8641  1.3341  1.1177
[torch.FloatTensor of size 5x3]

***
加算方法４：

In [11]:
y.add_(x)     # "add_(x)" -> y = y + x 
y


 0.4565  1.7649  0.5953
 1.7001  0.1802  0.9959
 1.0320  0.7855  0.3711
 1.2973  0.9961  0.7011
 0.8641  1.3341  1.1177
[torch.FloatTensor of size 5x3]

### Note
基本的に，演算の関数の後に `_` がついているものは代入の操作を表す。
上記の例では，
```Python
y.add_(x)
```
は，`y = y+x`を行うことと同じ。
***

`numpy`と同じように，次のようにインデックスを指定できる。

In [12]:
x[:, 1]


 0.8354
 0.0238
 0.2469
 0.2259
 0.8614
[torch.FloatTensor of size 5]

### あとで読みたい
`Tensor`の演算は他にも，インデックス，スライス，線形代数の演算などが100以上ある。詳細は[こちら](http://pytorch.org/docs/torch)。

***

## `Torch.Tensor`と`numpy`を行ったり来たり

`Torch.Tensor`と`numpy`の要素は，同じメモリの場所に格納されており，相互の変換は簡単にできる。

### `numpy`への変換

In [13]:
a = torch.ones(5)
a


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

In [14]:
b = a.numpy()
b

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

***
ここで，面白いことに…

In [16]:
a.add_(1)
print(a)
print(b)


 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

[ 2.  2.  2.  2.  2.]


***
### `torch.Tensor`に変換

In [18]:
import numpy as np

a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[ 2.  2.  2.  2.  2.]

 2
 2
 2
 2
 2
[torch.DoubleTensor of size 5]



CPU上の`Tensor`は，全て`numpy`に変換できるらしい（GPU上のTensorは変換できないということだろうか）。

## `autograd`と`Variable`

Autogradは”automatic differentiation”のことで，そのパッケージである`autograd`はPyTorchの中核を担っている。これから，`autograd`に少し触れてみて，それをもとにニューラルネットワークを学習させてみよう。

`autograd`パッケージは，Tensorの全ての演算に対して自動微分を行うためのもの。これは，PyTorchの特徴である"define-by-run"を実現している。つまり，順伝播のコードを書くだけで逆伝播が定義できる。

### `Variable`

- `autograd.Variable`が，`autograd`の中心的なパッケージである。`Variable`は`Tensor`のラッパーであり，`Tensor`のほぼ全ての演算が含まれている。
- ネットワークを定義してしまえば，`.backward()`を呼び出すだけで勾配計算を自動的に行うことができる。

Tensorの生データには`.data`でアクセスできる。そして，`Variable`に関する勾配は`.grad`に蓄積されている。`Variable`の概念図を以下に示す。

![概念図](http://pytorch.org/tutorials/_images/Variable.png)

### `Function`

autogradに関して，もうひとつ重要なクラスがあります。それは`Function`と呼ばれるパッケージです。
`Variable`と`Function`は内部でつながっていて，この２つによってニューラルネットワークのグラフが構築されます。そしてこのグラフに，ニューラルネットの計算の全ての履歴が残ります。

生成されたvariableのそれぞれに`.grad_fn`という属性があり，この属性によってどの`Function`によってvariableが生成されたのかを参照できる。ただし，ユーザによって作られたvariableの場合`grad_fn`は`None`となる。

variableの導関数を計算したいのであれば，variableがもっている`.backward()`を呼び出すと良い。もしvariableが単一の要素だけのとき，`backward()`には特に引数を指定する必要はないが，複数の要素をもつときは引数`grad_output`を指定してやる必要がある（←どういうこと？）。

と言っても分かりづらいので，具体例を交えながら見ていきましょう。

***

In [19]:
import torch
from torch.autograd import Variable

***
variableを作ってみます：

In [21]:
x = Variable(torch.ones(2, 2), requires_grad = True)
x

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]

***
次に，variableの加算をしてみましょう：

In [23]:
y = x + 2
y

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]

***
さっき言ったように，何らかの演算の後に生成されたvariableには属性`grad_fn`が付与されます。`y`の`grad_fn`を参照してみると：

In [25]:
y.grad_fn

***
`y`にもっと色んな演算をしていきます：

In [27]:
z = y * y * 3
out = z.mean()
print(z, out)

Variable containing:
 27  27
 27  27
[torch.FloatTensor of size 2x2]
 Variable containing:
 27
[torch.FloatTensor of size 1]



***
## 勾配計算

いよいよ逆伝播の計算です。
次に行う`out.backward()`は，`out.backward(torch.tensor([1.0]))`と等価です。

In [28]:
out.backward()

***
勾配を出力してみましょう。勾配とはすなわち $\frac{d({\rm out})}{dx}$　のことです。

In [29]:
x.grad

Variable containing:
 4.5000  4.5000
 4.5000  4.5000
[torch.FloatTensor of size 2x2]

`4.5`の要素をもつ行列（テンソル）が得られました。

これはどういうことなのでしょうか？

`out` Variable を"$o$"と呼ぶことにします。$o$は，次式で得られます。

$o = \frac{1}{4}\sum_i z_i$

ここで，$z_i = 3(x_i + 2)^2$ですから，

$\frac{\partial o}{\partial x_i}=\frac{3}{2}(x_i + 2)$

$x_i$の各要素は$1$なので，


$\frac{\partial o}{\partial x_i}|_{x_i = 1} = 4.5$

***

In [30]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


`forward`

In [31]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weighty

10
torch.Size([6, 1, 5, 5])


In [32]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
-0.0744  0.0788 -0.0589 -0.0867  0.0384  0.0192  0.1100  0.0702 -0.0357 -0.1127
[torch.FloatTensor of size 1x10]



In [33]:
net.zero_grad() 
out.backward(torch.randn(1, 10))